# Crimson Responses - kilosort

basically the same thing as the other crimson response stuff, except we're working with kilosort stuff

In [1]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from scipy import signal
from sklearn.decomposition import PCA
import os, glob, re
import openephys_utils 
from tqdm.notebook import tqdm
import kilosort

%matplotlib qt


Get firing rates from kilosort data -- since this will need to work differently from the version I did with the "spike dictionary"

In [37]:
def kilosort_FR(kilosort_dir:str, bin_sec:float = .01, fs:int = 30000):
    '''
    Return the firing rates for each template

    inputs:
        kilosort_dir:str        - directory where the kilosort results were stored
        bin_sec:float           - bin length in seconds [.01]
        fs:int                  - sampling rate [30000]

    outputs:
        firing_rates:np.array   - TxN array of firing rates of N templates
    '''
    spike_times = np.load(os.path.join(kilosort_dir, 'spike_times.npy'))
    spike_templates = np.load(os.path.join(kilosort_dir, 'spike_templates.npy'))

    # bin times defined by sample #s, not seconds
    bin_times = np.arange(np.ceil(spike_times[-1]), step=bin_sec*fs)
    # empty array to fill later
    firing_rates = np.ndarray((bin_times.shape[0]-1, np.unique(spike_templates).shape[0]))
    for template in np.unique(spike_templates): # for each template...
        firing_rates[:,template] = np.histogram(spike_times[spike_templates == template], bins=bin_times)[0]

    return firing_rates, bin_times[:-1]




    

In [ ]:
def prestim_mean_FR(firing_rates, stim_start):

In [ ]:
def fr_percentages():

In [ ]:
def response_depth_distance(firing_rate:np.array, stims:np.array):

batch run through kilosort on filtered data

october 2nd

In [ ]:
# get a list of directories
base_dir = 'Z:\\BrainPatch\\20241002\\lateral\\'
directories = [os.path.join(base_dir, directory) for directory in os.listdir(base_dir) if ('2ms' in directory and '15mA' in directory)]

# probe and settings
probe_name = 'Z:\\BrainPatch\\20241002\\64-4shank-poly-brainpatch-chanMap.mat'
settings = {'probe_name':probe_name,
            'n_chan_bin':64,
            'nearest_chans':1}

# sos for a BPF
filtered = True # should we filter the data?
sos = signal.butter(N = 8, Wn = [300, 6000], btype='bandpass', fs=30000, output='sos') if filtered else None

# binning decisions
bin_sec = 0.001

# to create a pandas array later I guess
stim_responses = [] # mean fr for 5 ms after stim
prestim_means = []

# length of interest for the responses and the "low mean firing rate"
resp_len = 5 # in bins
low_fr_cutoff = 1


for directory in tqdm(directories):
# for i_directory,directory in enumerate(directories):
#     if i_directory > 0:
#         continue

    # pull out the current and distance
    current = int(re.search('(\d{1,2})mA', directory)[1])
    distance = int(re.search('(\d{3,4})um', directory)[1])

    # print it all out
    print(f'{current}mA {distance}mm')

    # get sig_eraasr if it doesn't exist already
    if not os.path.exists(os.path.join(directory, 'sig_eraasr.npy')):
        sig, timestamps, stims, stim_ts = openephys_utils.open_sig_stims(directory)

        # clean, pull out threshold crossings, get firing rates
        sig_eraasr = openephys_utils.ERAASR(sig, save_dir=directory)
        # spike_dict = openephys_utils.threshold_crossings(sig_eraasr, multi_rejection=None, low_cutoff=-20)
        # fr, bins = openephys_utils.calc_FR(spike_dict, max_samp = sig_eraasr.shape[0], min_samp = 0, bin_sec = bin_sec)
    else:
        sig_eraasr = np.load(os.path.join(directory, 'sig_eraasr.npy'))

    # filter at 300, 6000
    if filtered:
        sig_filter = signal.sosfiltfilt(sos, sig_eraasr, axis=0)
        np.save(os.path.join(directory, 'sig_filter.npy'), sig_filter)


    # kilosort
    if not filtered:
        settings['filename'] = os.path.join(directory,'sig_eraasr.npy')
        results_dir=os.path.join(directory, 'kilosort4_unfiltered')
        if not os.path.exists(results_dir): # run it if it doesn't already exist. Otherwise just use the existing
            kilosort.run_kilosort(settings, file_object=sig_eraasr.astype(np.float32), data_dtype='float32', results_dir=results_dir)
        else:
            print(f'{results_dir} already exists, using existing files')

    else:
        settings['filename'] = os.path.join(directory,'sig_filter.npy')
        results_dir=os.path.join(directory, 'kilosort4_filtered')
        if not os.path.exists(results_dir): # run it if it doesn't already exist. Otherwise just use the existing
            kilosort.run_kilosort(settings, file_object=sig_eraasr.astype(np.float32), data_dtype='float32', results_dir=results_dir)
        else:
            print(f'{results_dir} already exists, using existing files')

    # firing rates from kilosort
    firing_rate, bins = kilosort_FR(results_dir, bin_sec=bin_sec)


    prestim_means = firing_rate[:int(stims[0,0]/(30000*bin_sec)), :].mean(axis=0)/bin_sec # put into firing rates instead of counts

    # get the means for 5 bins after the stims
    # indices for 5 ms after end of stimulation
    poststim_inds = np.array([np.arange(start=int(stim/(30000*bin_sec)), stop=int(stim/(30000*bin_sec)+resp_len)) for stim in stims[:,1]]).flatten()
    # split out that portion of the array and reshape it to a Stims x Time x Templates array
    poststim_resp = firing_rate[poststim_inds,:].reshape((stims.shape[0], resp_len, firing_rate.shape[1]))
    # find the mean
    poststim_mean = poststim_resp.mean(axis=0)/bin_sec # put into firing rates instead of counts

    # split into three colors -- minimal fr before, fr increases, fr decreases
    color_inds = 1 + (poststim_mean.mean(axis=0)>prestim_means).astype(int)
    color_inds[prestim_means < low_fr_cutoff] = 0

    # colors for depending on change of firing rate

    fig,ax = plt.subplots(nrows = 3)

    # colors = ['blue','red','black']
    for channel in np.arange(prestim_means.shape[0]):
        ax[color_inds[channel]].plot(poststim_mean[:,channel] - prestim_means[channel])


    ax[0].set_title('Low initial firing')
    ax[1].set_title('Decreased firing')
    ax[2].set_title('Increased firing')

    fig.savefig(os.path.join(directory, 'FR_changes.png'))




  0%|          | 0/7 [00:00<?, ?it/s]

15mA 400mm
Z:\BrainPatch\20241002\lateral\Crimson__2024-10-02_12-35-37__15mA_2ms_400um\kilosort4_filtered already exists, using existing files
15mA 400mm
saving data to Z:\BrainPatch\20241002\lateral\Crimson__2024-10-02_12-37-44__15mA_2ms_400um


kilosort.run_kilosort: Kilosort version 4.0.17
kilosort.run_kilosort: Kilosort version 4.0.17
kilosort.run_kilosort: Kilosort version 4.0.17
kilosort.run_kilosort: Sorting Z:\BrainPatch\20241002\lateral\Crimson__2024-10-02_12-37-44__15mA_2ms_400um\sig_filter.npy
kilosort.run_kilosort: Sorting Z:\BrainPatch\20241002\lateral\Crimson__2024-10-02_12-37-44__15mA_2ms_400um\sig_filter.npy
kilosort.run_kilosort: Sorting Z:\BrainPatch\20241002\lateral\Crimson__2024-10-02_12-37-44__15mA_2ms_400um\sig_filter.npy
kilosort.run_kilosort: ----------------------------------------
kilosort.run_kilosort: ----------------------------------------
kilosort.run_kilosort: ----------------------------------------
kilosort.run_kilosort: Using GPU for PyTorch computations. Specify `device` to change this.
kilosort.run_kilosort: Using GPU for PyTorch computations. Specify `device` to change this.
kilosort.run_kilosort: Using GPU for PyTorch computations. Specify `device` to change this.
kilosort.run_kilosort:  


15mA 400mm
saving data to Z:\BrainPatch\20241002\lateral\Crimson__2024-10-02_12-52-58__15mA_2ms_400um


kilosort.run_kilosort: Kilosort version 4.0.17
kilosort.run_kilosort: Kilosort version 4.0.17
kilosort.run_kilosort: Kilosort version 4.0.17
kilosort.run_kilosort: Kilosort version 4.0.17
kilosort.run_kilosort: Sorting Z:\BrainPatch\20241002\lateral\Crimson__2024-10-02_12-52-58__15mA_2ms_400um\sig_filter.npy
kilosort.run_kilosort: Sorting Z:\BrainPatch\20241002\lateral\Crimson__2024-10-02_12-52-58__15mA_2ms_400um\sig_filter.npy
kilosort.run_kilosort: Sorting Z:\BrainPatch\20241002\lateral\Crimson__2024-10-02_12-52-58__15mA_2ms_400um\sig_filter.npy
kilosort.run_kilosort: Sorting Z:\BrainPatch\20241002\lateral\Crimson__2024-10-02_12-52-58__15mA_2ms_400um\sig_filter.npy
kilosort.run_kilosort: ----------------------------------------
kilosort.run_kilosort: ----------------------------------------
kilosort.run_kilosort: ----------------------------------------
kilosort.run_kilosort: ----------------------------------------
kilosort.run_kilosort: Using GPU for PyTorch computations. Specify `

...

kilosort.run_kilosort: Kilosort version 4.0.17
kilosort.run_kilosort: Kilosort version 4.0.17
kilosort.run_kilosort: Kilosort version 4.0.17
kilosort.run_kilosort: Kilosort version 4.0.17
kilosort.run_kilosort: Kilosort version 4.0.17
kilosort.run_kilosort: Kilosort version 4.0.17
kilosort.run_kilosort: Sorting Z:\BrainPatch\20241002\lateral\Crimson__2024-10-02_13-39-44__15mA_2ms_900um\sig_filter.npy
kilosort.run_kilosort: Sorting Z:\BrainPatch\20241002\lateral\Crimson__2024-10-02_13-39-44__15mA_2ms_900um\sig_filter.npy
kilosort.run_kilosort: Sorting Z:\BrainPatch\20241002\lateral\Crimson__2024-10-02_13-39-44__15mA_2ms_900um\sig_filter.npy
kilosort.run_kilosort: Sorting Z:\BrainPatch\20241002\lateral\Crimson__2024-10-02_13-39-44__15mA_2ms_900um\sig_filter.npy
kilosort.run_kilosort: Sorting Z:\BrainPatch\20241002\lateral\Crimson__2024-10-02_13-39-44__15mA_2ms_900um\sig_filter.npy
kilosort.run_kilosort: Sorting Z:\BrainPatch\20241002\lateral\Crimson__2024-10-02_13-39-44__15mA_2ms_900um\s

15mA 1200mm
saving data to Z:\BrainPatch\20241002\lateral\Crimson__2024-10-02_14-05-20__15mA_2ms_1200um


kilosort.run_kilosort: Kilosort version 4.0.17
kilosort.run_kilosort: Kilosort version 4.0.17
kilosort.run_kilosort: Kilosort version 4.0.17
kilosort.run_kilosort: Kilosort version 4.0.17
kilosort.run_kilosort: Kilosort version 4.0.17
kilosort.run_kilosort: Kilosort version 4.0.17
kilosort.run_kilosort: Kilosort version 4.0.17
kilosort.run_kilosort: Sorting Z:\BrainPatch\20241002\lateral\Crimson__2024-10-02_14-05-20__15mA_2ms_1200um\sig_filter.npy
kilosort.run_kilosort: Sorting Z:\BrainPatch\20241002\lateral\Crimson__2024-10-02_14-05-20__15mA_2ms_1200um\sig_filter.npy
kilosort.run_kilosort: Sorting Z:\BrainPatch\20241002\lateral\Crimson__2024-10-02_14-05-20__15mA_2ms_1200um\sig_filter.npy
kilosort.run_kilosort: Sorting Z:\BrainPatch\20241002\lateral\Crimson__2024-10-02_14-05-20__15mA_2ms_1200um\sig_filter.npy
kilosort.run_kilosort: Sorting Z:\BrainPatch\20241002\lateral\Crimson__2024-10-02_14-05-20__15mA_2ms_1200um\sig_filter.npy
kilosort.run_kilosort: Sorting Z:\BrainPatch\20241002\la

15mA 1500mm
saving data to Z:\BrainPatch\20241002\lateral\Crimson__2024-10-02_14-22-56__15mA_2ms_1500um


kilosort.run_kilosort: Kilosort version 4.0.17
kilosort.run_kilosort: Kilosort version 4.0.17
kilosort.run_kilosort: Kilosort version 4.0.17
kilosort.run_kilosort: Kilosort version 4.0.17
kilosort.run_kilosort: Kilosort version 4.0.17
kilosort.run_kilosort: Kilosort version 4.0.17
kilosort.run_kilosort: Kilosort version 4.0.17
kilosort.run_kilosort: Kilosort version 4.0.17
kilosort.run_kilosort: Sorting Z:\BrainPatch\20241002\lateral\Crimson__2024-10-02_14-22-56__15mA_2ms_1500um\sig_filter.npy
kilosort.run_kilosort: Sorting Z:\BrainPatch\20241002\lateral\Crimson__2024-10-02_14-22-56__15mA_2ms_1500um\sig_filter.npy
kilosort.run_kilosort: Sorting Z:\BrainPatch\20241002\lateral\Crimson__2024-10-02_14-22-56__15mA_2ms_1500um\sig_filter.npy
kilosort.run_kilosort: Sorting Z:\BrainPatch\20241002\lateral\Crimson__2024-10-02_14-22-56__15mA_2ms_1500um\sig_filter.npy
kilosort.run_kilosort: Sorting Z:\BrainPatch\20241002\lateral\Crimson__2024-10-02_14-22-56__15mA_2ms_1500um\sig_filter.npy
kilosort.

In [7]:
openephys_utils.plot_peristim_FR(firing_rate, bins, stims)

In [10]:
post_stim_inds = np.array([np.arange(start=stim/(30000*bin_sec), stop=stim+.005*fs, step=fs)])

(array([], dtype=int64), array([], dtype=int64))

In [147]:

# stimulations prior to 

prestim_means = firing_rate[:int(stims[0,0]/(30000*bin_sec)), :].mean(axis=0)/bin_sec # put into firing rates instead of counts

resp_len = 30
low_fr_cutoff = 1

# get the means for 5 bins after the stims
# indices for 5 ms after end of stimulation
poststim_inds = np.array([np.arange(start=int(stim/(30000*bin_sec))-2, stop=int(stim/(30000*bin_sec)+resp_len-2)) for stim in stims[:,1]]).flatten()
# split out that portion of the array and reshape it to a Stims x Time x Templates array
poststim_resp = firing_rate[poststim_inds,:].reshape((stims.shape[0], resp_len, firing_rate.shape[1]))
# find the mean
poststim_mean = poststim_resp.mean(axis=0)/bin_sec # put into firing rates instead of counts

# split into three colors -- minimal fr before, fr increases, fr decreases
color_inds = 1 + (poststim_mean.mean(axis=0)>prestim_means).astype(int)
color_inds[prestim_means < low_fr_cutoff] = 0

# colors for depending on change of firing rate
# cmap = np.array(plt.colormaps.get_cmap('tab20').colors)[color_inds,:]

fig,ax = plt.subplots(nrows = 3)

# colors = ['blue','red','black']
for channel in np.arange(prestim_means.shape[0]):
    ax[color_inds[channel]].plot(poststim_mean[:,channel] - prestim_means[channel])


ax[0].set_title('Low initial firing')
ax[1].set_title('Decreased firing')
ax[2].set_title('Increased firing')



# poststim_mean = np.ndarray((5,prestim_means.shape[0]))
# poststim_std = np.ndarray((5,prestim_means.shape[0]))
# for channel in np.arange(prestim_means.shape[0]):
#     if prestim_means[channel] != 0:
#         poststim_mean[:,channel] = poststim_resp[:,:,channel].mean(axis=1)/prestim_means[channel]
#         poststim_std[:,channel] = poststim_resp[:,:,channel].std(axis=1)/prestim_means[channel]
#     else:
#         poststim_mean[:,channel] = 0
#         poststim_std[:,channel] = 0


Text(0.5, 1.0, 'Increased firing')

In [123]:
# cmap[channel,:]

array([0.17254902, 0.62745098, 0.17254902])

In [67]:
fig,ax = plt.subplots()

ax.plot(np.arange(5),poststim_mean)

In [60]:
prestim_means

array([0.02272727, 0.01704545, 0.00568182, 0.00757576, 0.00757576,
       0.        , 0.        , 0.02272727, 0.00757576, 0.        ,
       0.01325758, 0.00568182, 0.09659091, 0.        , 0.02272727,
       0.0094697 , 0.04356061, 0.        , 0.00189394, 0.125     ,
       0.03598485, 0.0530303 , 0.03409091, 0.        , 0.01515152,
       0.00189394, 0.00378788, 0.06439394, 0.03030303, 0.00189394,
       0.04166667, 0.        , 0.        , 0.        , 0.02272727,
       0.01704545, 0.0719697 , 0.04356061, 0.02651515, 0.00189394,
       0.13636364, 0.        , 0.00757576, 0.00189394, 0.03409091,
       0.02651515, 0.06818182, 0.02840909, 0.02840909, 0.01515152,
       0.        , 0.        , 0.03219697, 0.02272727, 0.00189394,
       0.14962121])